<a href="https://colab.research.google.com/github/dmbk/Anomaly-Detection-System/blob/master/ConvLSTM_WGAN_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install imageio
#!pip install tensorflow_datasets
!pip install keras-layer-normalization
from google.colab import drive
!pip install alive-progress
drive.mount('/content/drive', force_remount=True)


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import skimage
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
import time
from os.path import join
from os import listdir
from os.path import isfile, join, isdir

#import keras
import argparse
from os.path import dirname
import cv2
from google.colab.patches import cv2_imshow
	
import statistics


import shutil

import matplotlib.pyplot as plt

from alive_progress import alive_bar


import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers.merge import _Merge
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
from keras import backend as K
from functools import partial
from keras_layer_normalization import LayerNormalization


In [0]:
class Config:
    def __init__(self, data_dir_, cwdir_name_, data_set):
        self.data_set_name = data_set
        self.data_dir = data_dir_
        self.data_set_dir = join(self.data_dir, data_set)
        self.cwdir_name = cwdir_name_
        self.cwdir = join(self.data_dir,self.cwdir_name)
        self.run_data = join(self.cwdir, "training_gan")
        self.image_dir = join(self.run_data,self.data_set_name,"Test/")
        self.ref_model_dir = join(self.run_data, "ref_model")
        if not os.path.exists(self.cwdir):
            os.mkdir(self.cwdir)
            os.mkdir(self.run_data)
    
        if not os.path.exists(self.run_data):
            #shutil.rmtree(self.run_data)
            os.mkdir(self.run_data)
            os.makedirs(self.image_dir, exist_ok=True)
            os.mkdir(self.ref_model_dir)

        self.curr_epoch = 0
        self.DATASET_PATH = join(self.data_set_dir,"Train/")
        self.TEST_DIR = join(self.data_set_dir,"Test/")
        self.BATCH_SIZE = 2
        self.EPOCHS = 25
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_GAN")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_GAN")

        self.penaltyLambda = 10
        self.retrain = 0
        self.curr_epoch = 1
        self.dim1 = 10
        self.dim2 = 256
        self.dim3 = 256
        self.dim4 = 1
        self.n_critic = 5
        self.r_alpha = 0.2

    def reconfig(self, new_name, batch_size = 4, epochs = 5, retrain = 0, curr_epoch = 0):
        self.cwdir_name = new_name
        self.cwdir = join(self.data_dir, self.cwdir_name)
        self.run_data = join(self.cwdir, "training_gan")
        self.image_dir = join(self.run_data,self.data_set_name,"Test/")
        self.ref_model_dir = join(self.run_data, "ref_model")

        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_GAN")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_GAN")

        self.curr_epoch = 0
        self.retrain = retrain
        if retrain == 0:
            print("Configuring train from scratch")
            if not os.path.exists(self.cwdir):
                os.mkdir(self.cwdir)
                os.mkdir(self.run_data)
    
            if os.path.exists(self.run_data):
                shutil.rmtree(self.run_data)
                os.mkdir(self.run_data)
                os.makedirs(self.image_dir, exist_ok=True)
        else:
            print("Configuring retrain mode\n")
            self.final_gen_path = join(self.run_data, "generator_ep-"+str(curr_epoch)+".hdf5")
            self.final_disc_path = join(self.run_data, "discriminator_ep-"+str(curr_epoch)+".hdf5")
            self.final_gan_path = join(self.run_data, "gan_ep-"+str(curr_epoch)+".hdf5")
            self.curr_epoch = curr_epoch
            print("Last epoch saved : "+str(curr_epoch))

class ModelContainer:
    def __init__(self, y0, y1):
        self.generator = y0
        self.discriminator = y1

 
conf = Config(data_dir_="/content/drive/My Drive/", cwdir_name_="Conv2DLSTM_ADAE", data_set="UCSD_Anomaly_Dataset.v1p2/UCSDped1/") 


In [0]:
def get_clips_by_stride(stride, frames_list, sequence_size):

    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

def get_clips_list(seq_size):

    clips = []
    # loop over the training folders (Train000,Train001,..)
    for f in sorted(listdir(conf.DATASET_PATH)):
        directory_path = join(conf.DATASET_PATH, f)
        if isdir(directory_path):
            all_frames = []
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(directory_path)):
                img_path = join(directory_path, c)
                if str(img_path)[-3:] == "tif":
                    img = Image.open(img_path).resize((256, 256))
                    img = np.array(img, dtype=np.float32)
                    img = (img - 127.5)/127.5
                    #img = (2*(img - np.amin(img))/(np.amax(img) - np.amin(img)))-1
                    #img = np.array(img, dtype=np.float32) / 256.0
                    all_frames.append(img)
            # get the 32-frames sequences from the list of images after applying data augmentation
            for stride in range(1, 3):
                clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=seq_size))
    return clips


def get_single_test(single_test_path, sz):
    test = np.zeros(shape=(sz, conf.dim2, conf.dim3, conf.dim4))
    cnt = 0
    for f in sorted(listdir(single_test_path)):
        if str(join(single_test_path, f))[-3:] == "tif":
            img = Image.open(join(single_test_path, f)).resize((conf.dim2, conf.dim3))
            #cv2_imshow(np.array(img,dtype=np.float32))
            #cv2.waitKey(0)
            img = np.array(img, dtype=np.float32) / 256
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test

def get_test_sequences(test_case_dir, sz):
    test = get_single_test(join(conf.TEST_DIR,test_case_dir), sz)
    print("Test case loaded")
    sz = test.shape[0] - conf.dim1
    sequences = np.zeros((sz, conf.dim1, conf.dim2, conf.dim3, conf.dim4))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((conf.dim1, conf.dim2, conf.dim3, conf.dim4))
        for j in range(0, conf.dim1):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip
    return sequences

def convert_images_back(image):
    #return np.reshape(image,(256, 256))*256.0
    #print(image.shape)
    return np.reshape(image[:, :, 0],(image.shape[0], image.shape[1]))*127.5 + 127.5

def evaluate(sequences, models, test_case_dir, gen_only):

    # get the reconstruction cost of all the sequences
    InX = Input(shape=sequences.shape[1:])
    tmpModel = Model(inputs=InX, outputs=[models.generator.get_layer('convT5').output, models.generator.get_layer('sr').output] )
    reconstructed_sequences, sa= tmpModel.predict(sequences,batch_size=conf.BATCH_SIZE)
    sr = models.discriminator.predict(reconstructed_sequences,batch_size=conf.BATCH_SIZE)

    sz = sequences.shape[0]
    
    print("Test size:"+str(sz))
    if gen_only == 1:
        os.makedirs(join(conf.image_dir,test_case_dir), exist_ok=True)
        for i in range(0, sz):
            #print("sz "+str(i)+"\n")
            #cv2_imshow(np.reshape(reconstructed_sequences[i][2],(256, 256))*256)
            if i < 10:
                img_num = "00"+str(i)
            elif i < 100:
                img_num = "0"+str(i)
            else:
                img_num = str(i)    
        
            cv2.imwrite(join(conf.image_dir, test_case_dir,"gen_"+img_num+".jpg"), convert_images_back(reconstructed_sequences[i][6]))

    plt.plot(sr)
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.show()

    plt.plot(sa)
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.show()

In [83]:

def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)


def gradient_penalty_loss(y_true, y_pred, averaged_samples,
                          gradient_penalty_weight):
    gradients = K.gradients(y_pred, averaged_samples)[0]

    gradients_sqr = K.square(gradients)

    gradients_sqr_sum = K.sum(gradients_sqr,
                              axis=np.arange(1, len(gradients_sqr.shape)))

    gradient_l2_norm = K.sqrt(gradients_sqr_sum)

    gradient_penalty = gradient_penalty_weight * K.square(1 - gradient_l2_norm)

    return K.mean(gradient_penalty)




class RandomWeightedAverage(_Merge):

    def _merge_function(self, inputs):
        weights = K.random_uniform((conf.BATCH_SIZE, 1, 1, 1, 1))
        return (weights * inputs[0]) + ((1 - weights) * inputs[1])



def save_model(model, path):
    # serialize model to JSON
    model_json = model.to_json()
    with open(path+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(path+".h5")
    print("Saved model to disk : "+path)

def loead_model(path):
    # load json and create model
    json_file = open(path+".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json, custom_objects={'LayerNormalization': LayerNormalization})
    # load weights into new model
    loaded_model.load_weights(path+".h5")
    print("Loaded model from disk : "+path)
    return loaded_model

def load_models_obj(gen_path, dis_path):
    return ModelContainer(loadl_model(gen_path), load_model(dis_path))


def get_generator():
    if conf.retrain == 1 and os.path.isfile(conf.GEN_MODEL_PATH+".json"):
        model_gen=load_model(conf.GEN_MODEL_PATH)
        return model_gen

    #init = keras.initializers.RandomNormal(stddev=0.02)
    
    print("Loading generator model")

    #input_shape=[conf.dim1, conf.dim2, conf.dim3, conf.dim4]
    #X_input = Input(input_shape)
    model = Sequential()
    model.add(TimeDistributed(Conv2D(128, (11, 11), strides=4, kernel_initializer='he_normal', padding="same", name="convT1"), batch_input_shape=[None, conf.dim1, conf.dim2, conf.dim3, conf.dim4]))
    model.add(LayerNormalization())
    model.add(Activation("relu"))
    model.add(TimeDistributed(Conv2D(64, (5, 5), strides=2, kernel_initializer='he_normal', padding="same", name="convT2")))
    model.add(LayerNormalization())
    model.add(Activation("relu"))
    # # # # #
    model.add(ConvLSTM2D(64, (3, 3), kernel_initializer='he_normal', padding="same", return_sequences=True))
    model.add(LayerNormalization())
    model.add(ConvLSTM2D(32, (3, 3), kernel_initializer='he_normal', padding="same", return_sequences=True))
    model.add(LayerNormalization())
    model.add(ConvLSTM2D(64, (3, 3), kernel_initializer='he_normal', padding="same", return_sequences=True))
    model.add(LayerNormalization())
    # # # # #
    model.add(TimeDistributed(Conv2DTranspose(64, (5, 5), strides=2, kernel_initializer='he_normal', padding="same", name="convT3")))
    model.add(LayerNormalization())
    model.add(Activation("relu"))
    model.add(TimeDistributed(Conv2DTranspose(128, (11, 11), strides=4, kernel_initializer='he_normal', padding="same", name="convT4")))
    model.add(LayerNormalization())
    model.add(Activation("relu"))
    model.add(TimeDistributed(Conv2D(1, (11, 11), activation="tanh", kernel_initializer='he_normal',  padding="same", name="convT5"), name="convTD5"))

    model.summary(line_length=150)
    return model


def get_discriminator():
    if conf.retrain == 1 and os.path.isfile(conf.DIS_MODEL_PATH+".json"):
        model_disc=load_model(conf.DIS_MODEL_PATH)
        return model_disc
    print("Loading discriminator model")

    #init = keras.initializers.RandomNormal(stddev=0.02)

    #input_shape=[conf.dim1, conf.dim2, conf.dim3, conf.dim4]
    #X_input = Input(input_shape)
    #X_input = tf.keras.layers.InputLayer(batch_size=conf.BATCH_SIZE, input_shape=input_shape)(X_input)
    model = Sequential()
    model.add(TimeDistributed(Conv2D(128, (11, 11), strides=4, kernel_initializer='he_normal', padding="same"), batch_input_shape=[None, conf.dim1, conf.dim2, conf.dim3, conf.dim4]))
    model.add(LayerNormalization())
    model.add(LeakyReLU(0.2))

    model.add(TimeDistributed(Conv2D(64, (5, 5), strides=2, kernel_initializer='he_normal', padding="same")))
    model.add(LayerNormalization())
    model.add(LeakyReLU(0.2))
    # # # # #
    model.add(TimeDistributed(Conv2D(128, (3, 3), strides=2, kernel_initializer='he_normal', padding="same")))
    model.add(LayerNormalization())
    model.add(LeakyReLU(0.2))

    model.add(TimeDistributed(Conv2D(64, (3, 3), strides=2, kernel_initializer='he_normal', padding="same")))
    model.add(LayerNormalization())
    model.add(LeakyReLU(0.2))

    model.add(TimeDistributed(Conv2D(32, (3, 3), strides=2, kernel_initializer='he_normal', padding="same")))
    model.add(LayerNormalization())
    model.add(LeakyReLU(0.2))

    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='linear', name="sr"))
    model.summary(line_length=150)

    return model


def build_WGANgp(generator, discriminator):

    img_shape = [conf.dim1, conf.dim2, conf.dim3, conf.dim4]
    for layer in discriminator.layers:
        layer.trainable = False
    discriminator.trainable = False
    generator_input = Input(shape=img_shape)
    generator_layers = generator(generator_input)
    discriminator_layers_for_generator = discriminator(generator_layers)
    generator_model = Model(inputs=[generator_input],
                            outputs=[generator.get_layer("convTD5").output, discriminator_layers_for_generator])
    generator_model.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9),
                        loss=['mse',wasserstein_loss], loss_weights=[0, 1],)


    for layer in discriminator.layers:
        layer.trainable = True
    for layer in generator.layers:
        layer.trainable = False
    discriminator.trainable = True
    generator.trainable = False


    real_samples = Input(shape=img_shape)
    generator_input_for_discriminator = Input(shape=img_shape)
    generated_samples_for_discriminator = generator(generator_input_for_discriminator)
    discriminator_output_from_generator = discriminator(generated_samples_for_discriminator)
    discriminator_output_from_real_samples = discriminator(real_samples)

    averaged_samples = RandomWeightedAverage()([real_samples,
                                            generated_samples_for_discriminator])
    averaged_samples_out = discriminator(averaged_samples)


    partial_gp_loss = partial(gradient_penalty_loss,
                          averaged_samples=averaged_samples,
                          gradient_penalty_weight=conf.penaltyLambda)
    partial_gp_loss.__name__ = 'gradient_penalty'

    discriminator_model = Model(inputs=[real_samples,
                                    generator_input_for_discriminator],
                            outputs=[discriminator_output_from_real_samples,
                                     discriminator_output_from_generator,
                                     averaged_samples_out])

    discriminator_model.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9),
                            loss=[wasserstein_loss,
                                  wasserstein_loss,
                                  partial_gp_loss])
    

    print(generator_model.summary(line_length=50))
    print(discriminator_model.summary(line_length=50))
 
    return ModelContainer(generator_model, discriminator_model)
   
get_generator()
get_discriminator()
#get_generator_skip_conn()
#get_discriminator_skip_conn()

In [0]:
# create a line plot of loss for the gan and save to file
def plot_history(r_loss_list, f_loss_list, f_r_loss_list, penalty_list, d_loss_list, image_name, iteration):
	# plot history
    fig, ax = plt.subplots(5, 2, figsize=(8.27,11.69))
    for j in range(2):
        ax[0,j].plot(r_loss_list, label="r_los")
        ax[1,j].plot(f_loss_list, label="f_loss")
        ax[2,j].plot(f_r_loss_list, label="f-r_loss")
        ax[3,j].plot(penalty_list, label="penalty")
        ax[4,j].plot(d_loss_list, label="d_loss")
    for i in range(5):
        ax[i,0].set_xlim([0,100])
        ax[i,1].set_xlim([100,iteration])
        for j in range(2):
            ax[i,j].grid()
            ax[i,j].legend()

    plt.legend()
    plt.savefig(image_name)
    plt.close()
 
  

def reconstruct_batch(model, sequences, epoch, folder_name):

    sz = sequences.shape[0]
    #InX = Input(shape=sequences.shape[1:]) 
    #tmpModel = Model(inputs=InX, outputs=model.get_layer("convTD5").output)
    [reconstructed_sequences, sr] = model.predict(sequences,batch_size=conf.BATCH_SIZE)

    path = join(conf.run_data,folder_name, str(epoch)+"_epoch")
    os.makedirs(path, exist_ok=True)
    for i in range(0, sz):
        #cv2_imshow(np.reshape(reconstructed_sequences[i][2],(256, 256))*256)
        if i < 10:
            img_num = "00"+str(i)
        elif i < 100:
            img_num = "0"+str(i)
        else:
            img_num = str(i)
        print("Reconstructing : "+ str(reconstructed_sequences.shape))
        cv2.imwrite(join(path,"gen_"+img_num+".jpg"), convert_images_back(reconstructed_sequences[i][6]))
    
def batch_generator(X, batch_size = conf.BATCH_SIZE):
    indices = np.arange(len(X)) 
    batch=[]
    while True:
            for i in indices:
                batch.append(i)
                #print("i = "+str(i))
                if len(batch)==batch_size:
                    #print(batch_size)
                    yield X[batch]
                    batch=[]

def train(test_cases):
    generator = get_generator()
    discriminator = get_discriminator()
    # WGAN-gp Training Model
    models = build_WGANgp(generator, discriminator)
    X_train = np.array(get_clips_list(conf.dim1))

    positive_y = np.ones((conf.BATCH_SIZE, 1), dtype=np.float32)
    negative_y = -positive_y
    dummy_y = np.zeros((conf.BATCH_SIZE, 1), dtype=np.float32)
    
    for epoch in range(conf.curr_epoch, conf.EPOCHS+1):

        discriminator_loss = []
        generator_loss = []
        minibatches_size = conf.BATCH_SIZE * conf.n_critic
        with alive_bar(len(range(int(X_train.shape[0] // (conf.BATCH_SIZE * conf.n_critic))))) as bar:
            for i in range(int(X_train.shape[0] // (conf.BATCH_SIZE * conf.n_critic))):
                bar()
                time.sleep(1)
                discriminator_minibatches = X_train[i * minibatches_size:
                                                (i + 1) * minibatches_size]
                for j in range(conf.n_critic):
                    image_batch = discriminator_minibatches[j * conf.BATCH_SIZE:
                                                    (j + 1) * conf.BATCH_SIZE]
                    #noise = np.random.rand(conf.BATCH_SIZE, 100).astype(np.float32)
                    discriminator_loss.append(models.discriminator.train_on_batch(
                        [image_batch, image_batch],
                        [positive_y, negative_y, dummy_y]))
                generator_loss.append(models.generator.train_on_batch(image_batch, [image_batch, positive_y]))
        
        save_model(generator, (join(conf.run_data,"generator"+"_ep-"+str(epoch+1))))
        save_model(discriminator, (join(conf.run_data,"discriminator"+"_ep-"+str(epoch+1))))

        reconstruct_batch(models.generator, test_cases[0:10*conf.BATCH_SIZE], epoch+1, "Train_Reconstructs")
  
    save_model(generator, conf.GEN_MODEL_PATH)
    save_model(discriminator, conf.DIS_MODEL_PATH)
    
    return models



def get_model(new_cwdir, batch_size, epochs, test_cases, reconfig=0, reload=1, gen_path=conf.GEN_MODEL_PATH, dis_path=conf.DIS_MODEL_PATH, retrain=0, curr_epoch=0):
    if reconfig == 1:
        conf.reconfig(new_cwdir, batch_size, epochs, retrain, curr_epoch)
    if reload == 1 and os.path.isfile(gen_path) and os.path.isfile(dis_path) :
        print("Loading the trained model")
        models = load_models_obj(gen_path, dis_path)
        #model.load_weights(model_path)
    else :
        print("Loading the new model")
        models = train(test_cases)
    
    return models

In [85]:
#!rm training_gan/gan.hdf5 -rf
#!mkdir training_gan


test_cases_dir = "Test008"
test_cases = get_test_sequences(test_cases_dir, 200)
#model_path="/content/drive/My Drive/Conv2DLSTM_VanillaGAN/training_gan/gan_ep-2.hdf5"tf.math.divide
models = get_model(new_cwdir="Conv2DLSTM_WGAN_GP", batch_size=4, epochs=100, reload=1, test_cases=test_cases, reconfig=1)#, retrain=1, curr_epoch=39, model_path="/content/drive/My Drive/Conv2DLSTM_ADAE_ep50_b2_tanh/training_gan/gan_ep-10.hdf5")
evaluate(test_cases, models, test_cases_dir, 1)

ValueError: ignored